In [ ]:
from load_data_and_augmentations import *

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (14,7)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
tensor_transform = get_tensor_transform('Kinetics', True)
train_spat_transform = get_spatial_transform(2)
train_temp_transform = get_temporal_transform()
valid_spat_transform = get_spatial_transform(0)
valid_temp_transform = va.TemporalFit(size=16)

root_dir = '/media/scratch/astamoulakatos/centre_Ch2/'
df = pd.read_csv('../important_csvs/more_balanced_dataset/small_set_multi_class.csv')


In [ ]:
#df = df.sample(frac=0.03)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

In [ ]:
df_train = get_df(df, 50, True, False, False)
class_image_paths, end_idx = get_indices(df_train, root_dir)
#train_loader = get_loader(16, 64, end_idx, class_image_paths, train_temp_transform, train_spat_transform, tensor_transform, False, False)

In [ ]:
class_image_paths

In [ ]:
len(df_train)

In [ ]:
end_idx

In [ ]:
len(class_image_paths), len(end_idx)

In [ ]:
seq_length = 1

In [ ]:
indices = []
for i in range(len(end_idx) - 1):
    start = end_idx[i]
    end = end_idx[i + 1] - seq_length
    if start > end:
        pass
    else:
        indices.append(torch.arange(start, end))
indices = torch.cat(indices)
#self.indices = indices

In [ ]:
indices[0:20]

In [ ]:
indices = indices[torch.randperm(len(indices))]
t = iter(indices.tolist())

In [ ]:
indices

In [ ]:
len(indices)

In [ ]:
from imbalanced_sampler_3 import MultilabelBalancedRandomSampler

In [ ]:
labels = []
for i in class_image_paths:
    labels.append(i[2])

In [ ]:
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    # print list 
    for x in unique_list: 
        print(x)

In [ ]:
len(labels)

In [ ]:
len(indices)

In [ ]:
unique(labels)

In [ ]:
labels = np.array(labels)

In [ ]:
train_sampler = MultilabelBalancedRandomSampler(
    labels, indices, class_choice="least_sampled"
)

In [ ]:
dataset = MyDataset(
        image_paths = class_image_paths,
        seq_length = 1,
        temp_transform = valid_temp_transform,
        spat_transform = valid_spat_transform,
        tensor_transform = tensor_transform,
        length = len(train_sampler),
        lstm = False,
        oned = True)

In [ ]:
len(dataset)

In [ ]:
train_loader = DataLoader(
        dataset,
        batch_size = 100,
        sampler = train_sampler,
        drop_last = True,
        num_workers = 0)

In [ ]:
df_valid = get_df(df, 50, False, True, False)
class_image_paths, end_idx = get_indices(df_valid, root_dir)
valid_loader = get_loader(1, 100, end_idx, class_image_paths, valid_temp_transform, valid_spat_transform, tensor_transform, False, True)

In [ ]:
print("================ Training phase ===============")
for x, y in train_loader:
    print("Label counts per class:")
    sum_ = y.sum(axis=0)
    print(sum_) 
    print("Difference between min and max")
    print(max(sum_) - min(sum_))
    print("")
print("")


In [ ]:
show_batch(train_loader, 10, False)

In [ ]:
dataset[0][0].shape

In [ ]:
dataset[0][1].shape

In [ ]:
print("=============== Validation phase ==============")
for x, y in valid_loader:
    print("Label counts per class:")
    sum_ = y.sum(axis=0)
    print(sum_)
    print("Difference between min and max")
    print(max(sum_) - min(sum_))
    print("")
print("")


In [ ]:
show_batch(valid_loader, 10 , False)